## Establishing a connection between Databricks and Storage account

In [0]:
#mounting our bronze container
# dbutils.fs.mount(
#     source="wasbs://bronze@medallionstorageaccont.blob.core.windows.net",
#     mount_point="/mnt/bronze/",
#     extra_configs={
#         "fs.azure.account.key.medallionstorageaccont.blob.core.windows.net": dbutils.secrets.get('DatabricksScope','storageaccountkey')}
# )

True

In [0]:
#checking what is in our storage account
# dbutils.fs.ls('/mnt/bronze/')
 

[FileInfo(path='dbfs:/mnt/bronze/20251224/', name='20251224/', size=0, modificationTime=0)]

In [0]:
#checking the filepath directly
# dbutils.fs.ls('/mnt/bronze/20251224')

[FileInfo(path='dbfs:/mnt/bronze/20251224/SalesLT.Address.parquet', name='SalesLT.Address.parquet', size=35923, modificationTime=1766580850000),
 FileInfo(path='dbfs:/mnt/bronze/20251224/SalesLT.Customer.parquet', name='SalesLT.Customer.parquet', size=100518, modificationTime=1766580850000),
 FileInfo(path='dbfs:/mnt/bronze/20251224/SalesLT.CustomerAddress.parquet', name='SalesLT.CustomerAddress.parquet', size=21289, modificationTime=1766580856000),
 FileInfo(path='dbfs:/mnt/bronze/20251224/SalesLT.Product.parquet', name='SalesLT.Product.parquet', size=196575, modificationTime=1766580852000),
 FileInfo(path='dbfs:/mnt/bronze/20251224/SalesLT.ProductCategory.parquet', name='SalesLT.ProductCategory.parquet', size=3421, modificationTime=1766580853000),
 FileInfo(path='dbfs:/mnt/bronze/20251224/SalesLT.ProductDescription.parquet', name='SalesLT.ProductDescription.parquet', size=77633, modificationTime=1766580850000),
 FileInfo(path='dbfs:/mnt/bronze/20251224/SalesLT.ProductModel.parquet', 

In [0]:
#mounting our silver storage container
# dbutils.fs.mount(
#     source="wasbs://silver@medallionstorageaccont.blob.core.windows.net",
#     mount_point="/mnt/silver/",
#     extra_configs={
#         "fs.azure.account.key.medallionstorageaccont.blob.core.windows.net": dbutils.secrets.get('DatabricksScope','storageaccountkey')}
# )

True

In [0]:
#mounting our gold storage container
# dbutils.fs.mount(
#     source="wasbs://gold@medallionstorageaccont.blob.core.windows.net",
#     mount_point="/mnt/gold/",
#     extra_configs={
#         "fs.azure.account.key.medallionstorageaccont.blob.core.windows.net": dbutils.secrets.get('DatabricksScope','storageaccountkey')}
# )

True

In [0]:
# #testing to see if we can create a database
# spark.sql('create database testing')

In [0]:
#drop the database created for testing
# spark.sql('drop database testing')

DataFrame[]

## Establish a connection between Databricks and Data Factory on ADF

In [0]:
#call out our parameters
# fileName = dbutils.widgets.get("fileName")
# tableSchema = dbutils.widgets.get("table_schema")
# tableName = dbutils.widgets.get("table_name")

# # create database if it doesnt exist
# spark.sql(f'create database if not exists {tableSchema}')

# # create table if it doesnt exist on the database
# spark.sql(
#     """CREATE TABLE IF NOT EXISTS """+ tableSchema+ """."""+ tableName+ """
#         USING PARQUET 
#         LOCATION 'mnt/bronze/"""+ fileName+"""/"""+tableSchema+"""."""+ tableName+""".parquet'
#     """
# )

### Tutor's script above didnt work for me

In [0]:
#call out our parameters
fileName = dbutils.widgets.get("fileName")
tableSchema = dbutils.widgets.get("table_schema")
tableName = dbutils.widgets.get("table_name")

# create database if it doesnt exist
spark.sql(f"CREATE DATABASE IF NOT EXISTS {tableSchema}")

# create table if it doesnt exist on the database
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {tableSchema}.{tableName}
AS SELECT * FROM parquet.`/mnt/bronze/{fileName}/{tableSchema}.{tableName}.parquet`
""")



[FileInfo(path='dbfs:/mnt/bronze/20251224/SalesLT.Address.parquet', name='SalesLT.Address.parquet', size=35923, modificationTime=1766593378000),
 FileInfo(path='dbfs:/mnt/bronze/20251224/SalesLT.Customer.parquet', name='SalesLT.Customer.parquet', size=100518, modificationTime=1766593377000),
 FileInfo(path='dbfs:/mnt/bronze/20251224/SalesLT.CustomerAddress.parquet', name='SalesLT.CustomerAddress.parquet', size=21289, modificationTime=1766593374000),
 FileInfo(path='dbfs:/mnt/bronze/20251224/SalesLT.Product.parquet', name='SalesLT.Product.parquet', size=196575, modificationTime=1766593378000),
 FileInfo(path='dbfs:/mnt/bronze/20251224/SalesLT.ProductCategory.parquet', name='SalesLT.ProductCategory.parquet', size=3421, modificationTime=1766593375000),
 FileInfo(path='dbfs:/mnt/bronze/20251224/SalesLT.ProductDescription.parquet', name='SalesLT.ProductDescription.parquet', size=77633, modificationTime=1766593376000),
 FileInfo(path='dbfs:/mnt/bronze/20251224/SalesLT.ProductModel.parquet', 

###### I realized that my version worked because I asked Spark to read the actual Parquet file first, while the tutor’s version assumed the Parquet file was a folder Spark could automatically manage; in simple terms, Spark can only infer a schema when it opens a real file and looks inside it, but the tutor’s approach pointed Spark at a location as if it were a directory of data when it was really just a single file, so Spark couldn’t find anything to inspect, whereas my approach explicitly told Spark “read this file, see its structure, then create the table from what you read,” which is why mine succeeded and the tutor’s didn’t for me, which i found out after getting a cannot infer schema error.